**Google Colab Imports**

In [ ]:
pip install spotipy --upgrade

Requirement already up-to-date: spotipy in /usr/local/lib/python3.7/dist-packages (2.18.0)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

Mounted at /content/drive/


**Loading in data from https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks using pandas dataframe using the data csv**

In [ ]:
spotify_data = pd.read_csv('drive/MyDrive/Music Recommender System/data.csv')
spotify_data.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,main_artist
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920,NaN
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920,NaN
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920,NaN
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920,NaN
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920,NaN


**Playing with the spotify_data df in order to change the format of the column artists**

In [ ]:
spotify_data_artist = spotify_data['artists']
spotify_data_artist.head()[0]

"['Mamie Smith']"

In [ ]:
s = spotify_data_artist[0]
s = list(s)
s = s[2:len(s) - 2]
s = "".join(s)
print(s)

Mamie Smith


In [ ]:
artist_name_format = []
for i in range(len(spotify_data_artist)):
  s = spotify_data_artist[i]
  s = list(s)
  s = s[2:len(s) -2]
  s = "".join(s)
  artist_name_format.append(s)

print(artist_name_format[:50])

['Mamie Smith', "Screamin' Jay Hawkins", 'Mamie Smith', 'Oscar Velazquez', 'Mixe', 'Mamie Smith & Her Jazz Hounds', 'Mamie Smith', 'Mamie Smith & Her Jazz Hounds', 'Francisco Canaro', 'Meetya', 'Dorville', 'Francisco Canaro', 'Ka Koula', 'Justrock', 'Takis Nikolaou', 'Aggeliki Karagianni', 'Giorgos Katsaros', 'Francisco Canaro', 'Giorgos Katsaros', 'Amalia Vaka', 'Ron van der Kraan', "Vasílis Besíris 'Tourkovasílis'", 'Tr0ma', 'Ron van der Kraan', 'Oscar Velazquez', 'Jodoli', 'Athanasía Alexandropoúlou', 'Dick Haymes', 'ST', 'Manolis Karapiperis', 'Félix Mayol', 'Meetya', 'Maurice Chevalier', 'David Daniells', "The Swingin' Bachelors", 'Ron van der Kraan', "The Swingin' Bachelors", 'Ron van der Kraan', 'Meetya', 'biex', "Mamie Smith', 'The Harlem Trio", 'Esther Lekain', 'Mamie Smith & Her Jazz Hounds', "Broke God', 'Pcxix", 'Two Charming Men', 'Tr0ma', "The Swingin' Bachelors", "The Swingin' Bachelors", "ST', '98', 'Niklas O", 'Aryiró Yiannopoúlou']


In [ ]:
spotify_data['formatted_artist'] = artist_name_format
spotify_data.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,main_artist,formatted_artist
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920,NaN,Mamie Smith
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920,NaN,Screamin' Jay Hawkins
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920,NaN,Mamie Smith
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920,NaN,Oscar Velazquez
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920,NaN,Mixe


**Loading in data from https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks using pandas dataframe using the data_w_genres csv**

In [ ]:
spotify_genre_data = pd.read_csv("drive/MyDrive/Music Recommender System/data_w_genres.csv")
spotify_genre_data.head()

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres
0,"""Cats"" 1981 Original London Cast",0.598500,0.470100,267072.000000,0.376203,0.010261,0.283050,-14.434300,0.209150,114.128800,0.358320,38.200000,5,1,10,['show tunes']
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,31.538462,5,1,26,[]
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.571429,0,1,7,[]
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.407407,0,1,27,[]
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,42.000000,5,1,7,[]


**Playing with spotify_genre_data df in order to format the genres column**

In [ ]:
spotify_genre = spotify_genre_data['genres']
spotify_genre.head()

0    ['show tunes']
1                []
2                []
3                []
4                []
Name: genres, dtype: object

In [ ]:
s = spotify_genre[8]
s = s.replace("'", '')
s = s.replace(" ", '')
s = list(s)
s = s[1:len(s) - 1]
s = "".join(s)
s = s.split(",")
print(s)
print(type(s))
print(s[0])
print(type(s[0]))

['comedyrock', 'comic', 'parody']
<class 'list'>
comedyrock
<class 'str'>


In [ ]:
artist_genre_format = []
for i in range(len(spotify_genre)):
  s = spotify_genre[i]
  s = s.replace("'", '')
  s = s.replace(" ", '')
  s = list(s)
  s = s[1:len(s) - 1]
  s = "".join(s)
  s = s.split(",")
  artist_genre_format.append(s)

print(artist_genre_format[:10])

[['showtunes'], [''], [''], [''], [''], [''], [''], [''], ['comedyrock', 'comic', 'parody'], ['']]


In [ ]:
spotify_genre_data['formatted_genre'] = artist_genre_format
spotify_genre_data.head()

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,genres,formatted_genre
0,"""Cats"" 1981 Original London Cast",0.598500,0.470100,267072.000000,0.376203,0.010261,0.283050,-14.434300,0.209150,114.128800,0.358320,38.200000,5,1,10,['show tunes'],[showtunes]
1,"""Cats"" 1983 Broadway Cast",0.862538,0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,31.538462,5,1,26,[],[]
2,"""Fiddler On The Roof” Motion Picture Chorus",0.856571,0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.571429,0,1,7,[],[]
3,"""Fiddler On The Roof” Motion Picture Orchestra",0.884926,0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.407407,0,1,27,[],[]
4,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.510714,0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,42.000000,5,1,7,[],[]


**Playing with both dfs to get all the songs with the artist "Cats" 1981 Original London Cast**

In [ ]:
spotify_genre_artists = spotify_genre_data['artists']
spotify_genre_artists.head()

0                     "Cats" 1981 Original London Cast
1                            "Cats" 1983 Broadway Cast
2          "Fiddler On The Roof” Motion Picture Chorus
3       "Fiddler On The Roof” Motion Picture Orchestra
4    "Joseph And The Amazing Technicolor Dreamcoat"...
Name: artists, dtype: object

In [ ]:
s = spotify_genre_artists[1]
cats = spotify_data[spotify_data['formatted_artist'].str.contains(s)]
cats.head(1)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,main_artist,formatted_artist
31997,0.906,"['Andrew Lloyd Webber', '""Cats"" 1983 Broadway ...",0.254,320573,0.231,0,2Qw6jgFXAmCBjTgpu18Nv2,0.0084,3,0.114,-9.668,1,Memory,44,1983-01-01,0.0302,81.509,0.0735,1983,NaN,"Andrew Lloyd Webber', '""Cats"" 1983 Broadway Ca..."


**Using the spotify API in order to get data from the user playlist we created**

In [ ]:
cid = 'e2a33ce0d8e94b5383609f0e5aee0ffb' 
secret = '59ae1a2b60054c94b24351e69fa74a90'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

playlist = sp.user_playlist('userPlaylist', '0qT5fFxIhEemCptwD1joGj') # playlist we created 

playlist_ids = [] # array that is going to hold all the track ids that are in our playlist
for item in playlist['tracks']['items']:
        track = item['track']
        playlist_ids.append(track['id'])
        
tracks = []
for i in range(len(playlist_ids)):
  curr = []

  track_info = sp.track(playlist_ids[i]) # returns a single track information about the track in a dictionary 
  track_audio_info = sp.audio_features(playlist_ids[i]) # returns a single track information about the audio in a dictionary 

  name = track_info ['name']
  artist = track_info ['album']['artists'][0]['name']
  release_date = track_info ['album']['release_date'][0:4]
  acousticness = track_audio_info[0]['acousticness']
  danceability = track_audio_info[0]['danceability']
  energy = track_audio_info[0]['energy']
  instrumentalness = track_audio_info[0]['instrumentalness']
  liveness = track_audio_info[0]['liveness']
  loudness = track_audio_info[0]['loudness']
  
  speechiness = track_audio_info[0]['speechiness']
  tempo = track_audio_info[0]['tempo']
  valence = track_audio_info[0]['valence']

  curr = [name, artist, release_date, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, valence]
  tracks.append(curr)

spotify_user_playlist = pd.DataFrame(tracks, columns = ['name', 'artist', 'release_date', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'])

In [ ]:
spotify_user_playlist.head()

,name,artist,release_date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,WITHOUT YOU,The Kid LAROI,2020,0.2130,0.662,0.413,0.000000,0.1340,-7.357,0.0299,93.005,0.467
1,TELL ME WHY,The Kid LAROI,2020,0.1090,0.600,0.723,0.000000,0.1670,-4.928,0.1580,149.881,0.463
2,All Girls Are The Same,Juice WRLD,2018,0.0769,0.671,0.529,0.000335,0.0856,-7.226,0.3070,161.989,0.203
3,Bandit (with YoungBoy Never Broke Again),Juice WRLD,2019,0.0687,0.474,0.631,0.000000,0.1320,-5.884,0.3430,180.051,0.425
4,Righteous,Juice WRLD,2020,0.0417,0.706,0.625,0.000000,0.1290,-7.426,0.0314,142.948,0.225


**Playing with spotify_genre_data df and spotify_user_playlist df in order to add the genres of each artist to each song in spotify_user_playlist**

In [ ]:
spotify_playlist_artists = spotify_user_playlist['artist']
art = []
for artist in spotify_playlist_artists:
  if artist not in art:
    art.append(artist)
print(len(art))
spotify_playlist_artists.head()

25


0    The Kid LAROI
1    The Kid LAROI
2       Juice WRLD
3       Juice WRLD
4       Juice WRLD
Name: artist, dtype: object

In [ ]:
playlist_artist_genre = []
for i in range(len(spotify_playlist_artists)):
  for j in range(len(spotify_genre_data.artists)):
    if(spotify_genre_data.artists.iloc[j] == spotify_playlist_artists[i]):
      playlist_artist_genre.append(spotify_genre_data.formatted_genre.iloc[j])

In [ ]:
print(playlist_artist_genre)
len(playlist_artist_genre)

[['australianhiphop'], ['australianhiphop'], ['chicagorap', 'melodicrap'], ['chicagorap', 'melodicrap'], ['chicagorap', 'melodicrap'], ['chicagorap', 'melodicrap'], ['chicagodrill', 'chicagorap', 'drill', 'rap', 'southernhiphop', 'trap'], ['chicagorap', 'melodicrap'], ['canadiancontemporaryr&b', 'canadianpop', 'pop'], ['canadiancontemporaryr&b', 'canadianpop', 'pop'], ['australianhiphop'], ['canadiancontemporaryr&b', 'canadianpop', 'pop'], ['canadiancontemporaryr&b', 'canadianpop', 'pop'], ['canadiancontemporaryr&b', 'canadianpop', 'pop'], ['dancepop', 'pop', 'poprap', 'post-teenpop', 'urbancontemporary'], ['atlhiphop', 'dancepop', 'dirtysouthrap', 'gangsterrap', 'hiphop', 'poprap', 'rap', 'southernhiphop', 'trap'], ['dancepop', 'pop', 'popdance', 'poprap'], ['canadianhiphop', 'canadianpop', 'hiphop', 'poprap', 'rap', 'torontorap'], ['canadianhiphop', 'canadianpop', 'hiphop', 'poprap', 'rap', 'torontorap'], ['canadianhiphop', 'canadianpop', 'hiphop', 'poprap', 'rap', 'torontorap'], ['c

50

In [ ]:
spotify_user_playlist['formatted_genre'] = playlist_artist_genre
spotify_user_playlist.head(5)

,name,artist,release_date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,formatted_genre
0,WITHOUT YOU,The Kid LAROI,2020,0.2130,0.662,0.413,0.000000,0.1340,-7.357,0.0299,93.005,0.467,[australianhiphop]
1,TELL ME WHY,The Kid LAROI,2020,0.1090,0.600,0.723,0.000000,0.1670,-4.928,0.1580,149.881,0.463,[australianhiphop]
2,All Girls Are The Same,Juice WRLD,2018,0.0769,0.671,0.529,0.000335,0.0856,-7.226,0.3070,161.989,0.203,"[chicagorap, melodicrap]"
3,Bandit (with YoungBoy Never Broke Again),Juice WRLD,2019,0.0687,0.474,0.631,0.000000,0.1320,-5.884,0.3430,180.051,0.425,"[chicagorap, melodicrap]"
4,Righteous,Juice WRLD,2020,0.0417,0.706,0.625,0.000000,0.1290,-7.426,0.0314,142.948,0.225,"[chicagorap, melodicrap]"




```
# This is formatted as code
```

**Reformatting the artist column in our spotify_user_playlist into a list to use TFIDF**

In [ ]:
name_list = []

for i in range(len(spotify_user_playlist["artist"])):
  new_array = []
  name = spotify_user_playlist["artist"][i]
  new_array.append(name.replace(" ", ''))
  name_list.append(new_array)
print(name_list)
spotify_user_playlist["artist_list"] = name_list
spotify_user_playlist.head()

[['TheKidLAROI'], ['TheKidLAROI'], ['JuiceWRLD'], ['JuiceWRLD'], ['JuiceWRLD'], ['JuiceWRLD'], ['GHerbo'], ['JuiceWRLD'], ['TheWeeknd'], ['TheWeeknd'], ['TheKidLAROI'], ['TheWeeknd'], ['TheWeeknd'], ['TheWeeknd'], ['JaySean'], ['T.I.'], ['BlackEyedPeas'], ['Drake'], ['Drake'], ['Drake'], ['Drake'], ['Drake'], ['BadBunny'], ['SwaeLee'], ['Drake'], ['InternetMoney'], ['JustinBieber'], ['LilTjay'], ['TravisScott'], ['PopSmoke'], ['DonToliver'], ['PopSmoke'], ['PoloG'], ['RoddyRicch'], ['LilTjay'], ['LilUziVert'], ['Future'], ['PopSmoke'], ['StaySolidRocky'], ['DaBaby'], ['Mustard'], ['DonToliver'], ['ChrisBrown'], ['LilUziVert'], ['LilUziVert'], ['PopSmoke'], ['TravisScott'], ['LilTjay'], ['ChrisBrown'], ['LilTecca']]


,name,artist,release_date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,formatted_genre,artist_list
0,WITHOUT YOU,The Kid LAROI,2020,0.2130,0.662,0.413,0.000000,0.1340,-7.357,0.0299,93.005,0.467,[australianhiphop],[TheKidLAROI]
1,TELL ME WHY,The Kid LAROI,2020,0.1090,0.600,0.723,0.000000,0.1670,-4.928,0.1580,149.881,0.463,[australianhiphop],[TheKidLAROI]
2,All Girls Are The Same,Juice WRLD,2018,0.0769,0.671,0.529,0.000335,0.0856,-7.226,0.3070,161.989,0.203,"[chicagorap, melodicrap]",[JuiceWRLD]
3,Bandit (with YoungBoy Never Broke Again),Juice WRLD,2019,0.0687,0.474,0.631,0.000000,0.1320,-5.884,0.3430,180.051,0.425,"[chicagorap, melodicrap]",[JuiceWRLD]
4,Righteous,Juice WRLD,2020,0.0417,0.706,0.625,0.000000,0.1290,-7.426,0.0314,142.948,0.225,"[chicagorap, melodicrap]",[JuiceWRLD]


**Refortmating the release_date column in our spotify_user_playlist into a list to use TFIDF**

In [ ]:
date_list = []
for i in range(len(spotify_user_playlist["release_date"])):
  new_array = []
  name = spotify_user_playlist["release_date"][i]
  new_array.append(name.replace(" ", ''))
  date_list.append(new_array)
print(name_list)
spotify_user_playlist["date_list"] = date_list
spotify_user_playlist.head()

[['TheKidLAROI'], ['TheKidLAROI'], ['JuiceWRLD'], ['JuiceWRLD'], ['JuiceWRLD'], ['JuiceWRLD'], ['GHerbo'], ['JuiceWRLD'], ['TheWeeknd'], ['TheWeeknd'], ['TheKidLAROI'], ['TheWeeknd'], ['TheWeeknd'], ['TheWeeknd'], ['JaySean'], ['T.I.'], ['BlackEyedPeas'], ['Drake'], ['Drake'], ['Drake'], ['Drake'], ['Drake'], ['BadBunny'], ['SwaeLee'], ['Drake'], ['InternetMoney'], ['JustinBieber'], ['LilTjay'], ['TravisScott'], ['PopSmoke'], ['DonToliver'], ['PopSmoke'], ['PoloG'], ['RoddyRicch'], ['LilTjay'], ['LilUziVert'], ['Future'], ['PopSmoke'], ['StaySolidRocky'], ['DaBaby'], ['Mustard'], ['DonToliver'], ['ChrisBrown'], ['LilUziVert'], ['LilUziVert'], ['PopSmoke'], ['TravisScott'], ['LilTjay'], ['ChrisBrown'], ['LilTecca']]


,name,artist,release_date,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,formatted_genre,artist_list,date_list
0,WITHOUT YOU,The Kid LAROI,2020,0.2130,0.662,0.413,0.000000,0.1340,-7.357,0.0299,93.005,0.467,[australianhiphop],[TheKidLAROI],[2020]
1,TELL ME WHY,The Kid LAROI,2020,0.1090,0.600,0.723,0.000000,0.1670,-4.928,0.1580,149.881,0.463,[australianhiphop],[TheKidLAROI],[2020]
2,All Girls Are The Same,Juice WRLD,2018,0.0769,0.671,0.529,0.000335,0.0856,-7.226,0.3070,161.989,0.203,"[chicagorap, melodicrap]",[JuiceWRLD],[2018]
3,Bandit (with YoungBoy Never Broke Again),Juice WRLD,2019,0.0687,0.474,0.631,0.000000,0.1320,-5.884,0.3430,180.051,0.425,"[chicagorap, melodicrap]",[JuiceWRLD],[2019]
4,Righteous,Juice WRLD,2020,0.0417,0.706,0.625,0.000000,0.1290,-7.426,0.0314,142.948,0.225,"[chicagorap, melodicrap]",[JuiceWRLD],[2020]


**TF-IDF**

1.   looking at using genre and artist name
2.   Item profiles = acousticness, danceability, instrumentalness, liveness, loudness, popularity,tempo, year

**TFIDF Values for The genre in our spotify_user_playlist**

---



In [ ]:
genrew = TfidfVectorizer(lowercase = False)
genrey = genrew.fit_transform(spotify_user_playlist["formatted_genre"].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(genrey.toarray())
genre_df.head()
genre_df.columns = [i for i in genrew.get_feature_names()]

name = []
for col in genre_df.columns:
    name.append(col)

sums = genre_df.sum()
genreArr = [sum for sum in sums]

tupleData = list(zip(name, sums)) 

genre_tfidf = pd.DataFrame(tupleData, columns=["genre", "value"])
genre_tfidf = genre_tfidf.sort_values(by ='value', ascending = False, ignore_index = True)
genre_tfidf.head()

,genre,value
0,rap,9.704539
1,melodicrap,8.309059
2,trap,6.655679
3,brooklyndrill,5.698932
4,poprap,5.577448


**TFIDF Values for The artists in our spotify_user_playlist**

In [ ]:
artistw = TfidfVectorizer(lowercase=False)
artisty = artistw.fit_transform(spotify_user_playlist["artist_list"].apply(lambda x: " ".join(x)))
artist_df = pd.DataFrame(artisty.toarray())
#print(artist_df)
artist_df.columns = [i for i in artistw.get_feature_names()]

artist = []
for col in artist_df.columns:
  artist.append(col)

sums = artist_df.sum()
artistArr = [sum for sum in sums ]

factor = [sum/len(spotify_user_playlist) for sum in sums]

tupleData = list(zip(artist, artistArr)) 

artist_tfidf = pd.DataFrame(tupleData, columns=["artist", "value"])
artist_tfidf["factor"] = factor

artist_tfidf = artist_tfidf.sort_values(by ='factor', ascending = False, ignore_index = True)

artist_tfidf.head(5)

,artist,value,factor
0,Drake,6.0,0.12
1,TheWeeknd,5.0,0.10
2,JuiceWRLD,5.0,0.10
3,PopSmoke,4.0,0.08
4,TheKidLAROI,3.0,0.06


**Getting a weighted Average Vector based on all the chosen song features in the user_spotify_playlist**

In [ ]:
# Weighted Average of Songs in Playlist

weighted_avg = []
acousticness = 0.0
danceability = 0.0
energy = 0.0
instrumentalness = 0.0
liveness = 0.0
loudness = 0.0
speechiness = 0.0
tempo = 0.0
valence = 0.0
for i in range(len(spotify_user_playlist)):
  acousticness += spotify_user_playlist['acousticness'][i]
  danceability += spotify_user_playlist['danceability'][i]
  energy += spotify_user_playlist['energy'][i]
  instrumentalness += spotify_user_playlist['instrumentalness'][i]
  liveness += spotify_user_playlist['liveness'][i]
  loudness += spotify_user_playlist['loudness'][i]
  speechiness += spotify_user_playlist['speechiness'][i]
  tempo += spotify_user_playlist['tempo'][i]
  valence += spotify_user_playlist['valence'][i]

acousticness /= len(spotify_user_playlist)
danceability /= len(spotify_user_playlist)
energy /= len(spotify_user_playlist)
instrumentalness /= len(spotify_user_playlist)
liveness /= len(spotify_user_playlist)
loudness /= len(spotify_user_playlist)
speechiness /= len(spotify_user_playlist)
tempo /= len(spotify_user_playlist)
valence /= len(spotify_user_playlist)

weighted_avg.append([acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence])
print(weighted_avg)

[[0.2074802, 0.7066999999999999, 0.59894, 0.0088669678, 0.20052199999999998, -6.78484, 0.14547799999999997, 121.02301999999997, 0.4250900000000001]]


**Get two lists (top genres and artists in playlist) based on the TFIDF Scores calcuated**




In [ ]:
#This code is to get the lists based on the tfid
genre_to_look_for = []
for i in range(len(genre_tfidf.value)):
  if(genre_tfidf.value[i] >= 5.2):
    genre_to_look_for.append(genre_tfidf.genre[i])
print(genre_to_look_for)

artists_to_look_for = []
for i in range(len(artist_tfidf.factor)):
  if(artist_tfidf.factor[i] >= 0.05):
    artists_to_look_for.append(artist_tfidf.artist[i])
print(artists_to_look_for)


['rap', 'melodicrap', 'trap', 'brooklyndrill', 'poprap', 'chicagorap']
['Drake', 'TheWeeknd', 'JuiceWRLD', 'PopSmoke', 'TheKidLAROI', 'LilTjay', 'LilUziVert']


**Get all the artists that match the genres found in genre_to_look_for in spotify_genre_data df**

In [ ]:
#This code is to get all the songs that fit the genre list above
all_artist_fits2 = []
for i in range(len(spotify_genre_data.formatted_genre)):
  for j in range(len(spotify_genre_data.formatted_genre[i])):
    if(spotify_genre_data.artists[i] not in all_artist_fits2):
      if(spotify_genre_data.formatted_genre[i][j] in genre_to_look_for):
        all_artist_fits2.append(spotify_genre_data.artists[i])

all_songs2 = spotify_data[spotify_data["formatted_artist"] == all_artist_fits2[0]]
                         
for i in range(1, len(all_artist_fits2)):
  try:
    all_songs2 = all_songs2.append(spotify_data[spotify_data["formatted_artist"] == all_artist_fits2[i]], ignore_index=True)
  except KeyError: 
    pass

all_songs2 = all_songs2[(all_songs2['release_date'] >= "2020-01-01") & (all_songs2['release_date'] <= "2020-12-31")]
all_songs2 = all_songs2.reset_index()
all_songs2.head()

,index,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,main_artist,formatted_artist
0,15,0.0679,['2 Chainz'],0.897,171838,0.767,1,6jzW19SQaLj1kLArvipGQZ,0.000364,11,0.1290,-4.544,1,Grey Area,73,2020-11-13,0.0959,134.913,0.675,2020,NaN,2 Chainz
1,207,0.4730,['A Boogie Wit da Hoodie'],0.644,161765,0.587,1,6cZH4rX1KTt1aJ3Ql6Ynja,0.000000,7,0.0759,-6.736,0,Me and My Guitar,76,2020-02-14,0.3530,173.972,0.680,2020,NaN,A Boogie Wit da Hoodie
2,312,0.2520,['Ace Hood'],0.752,200655,0.525,1,5dv4YSgkpflUQ31vIW9qXg,0.000000,10,0.1150,-9.034,0,We Ball,69,2020-05-29,0.2480,154.961,0.151,2020,NaN,Ace Hood
3,384,0.1980,['Baby Keem'],0.770,156774,0.636,1,02iYJG3KLBJODa5JkQ4O6y,0.000003,10,0.1150,-7.636,0,hooligan,69,2020-09-18,0.0901,93.032,0.414,2020,NaN,Baby Keem
4,523,0.0200,['Big Sean'],0.765,197760,0.695,1,5MeRuGMPReNSIGNfQf9t3y,0.043700,2,0.1300,-6.570,1,Dance (A$$),9,2020-09-25,0.3420,159.865,0.700,2020,NaN,Big Sean


**Create a matrix of all the songs from all_songs2 df where each row contains the song features**

In [ ]:
s2_labels = []
for i in range(all_songs2.shape[0]):
  current = []
  for column in all_songs2:
    if (column == "acousticness" or column == "danceability" or column == "energy" or column == "instrumentalness" or column == "liveness" or column == "loudness" or column == "speechiness" or column == "tempo" or column == "valence" ):
      current.append(all_songs2[column][i])
  s2_labels.append(current)
print(len(s2_labels))

206


**Recommend 10 songs using the weighted_avg vector and the matrix s2_labels by using cosine_similarit (Recommendation by genre)y**

In [ ]:
recommendations2 = []
temp = cosine_similarity(weighted_avg, s2_labels)
r2df = pd.DataFrame(temp).T # transpose row and comlums swap
top10 = r2df.nlargest(r2df.shape[0],0).index # index of top 10 closeest to songs in playlist
for index in top10:
  recommendations2.append("'" + all_songs2['name'][index] + "'" + " by " + all_songs2['artists'][index][2:-2])

chosenR2 = []
print("You should listen to: ")
i = 1
while len(chosenR2) < 10:
  ind = random.randint(0, r2df.shape[0])
  try:
    if recommendations2[ind] not in chosenR2:
      chosenR2.append(recommendations2[ind])
      print(i,": " + recommendations2[ind])
      i += 1
  except IndexError:
    pass

You should listen to: 
1 : 'Good News' by Mac Miller
2 : 'bloody valentine' by Machine Gun Kelly
3 : 'Said Sum' by Moneybagg Yo
4 : 'HOLIDAY' by Lil Nas X
5 : 'Take You Dancing' by Jason Derulo
6 : 'Whole Lotta Choppas' by Sada Baby
7 : 'Meh' by Playboi Carti
8 : 'Tyler Herro' by Jack Harlow
9 : 'BEST INTEREST' by Tyler, The Creator
10 : 'Baby Pluto' by Lil Uzi Vert


**Fromatting spotify_data df in order to compare each artist to the artists_to_look_for list**

In [ ]:
joinedArtists = []
for i in range(len(spotify_data.formatted_artist)):
  name = spotify_data.formatted_artist[i]
  name = name.replace(" ", "")
  joinedArtists.append(name)
spotify_data["joined_artist"] = joinedArtists
spotify_data.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,main_artist,formatted_artist,joined_artist
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920,NaN,Mamie Smith,MamieSmith
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920,NaN,Screamin' Jay Hawkins,Screamin'JayHawkins
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920,NaN,Mamie Smith,MamieSmith
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920,NaN,Oscar Velazquez,OscarVelazquez
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920,NaN,Mixe,Mixe


**Get all the songs that match the artist found in the spotify_data df into a new df called all_songs3**


In [ ]:
all_songs3 = spotify_data[spotify_data["joined_artist"] == artists_to_look_for[0]]
all_songs3.head()                
for i in range(1, len(artists_to_look_for)):
  try:
    all_songs3 = all_songs3.append(spotify_data[spotify_data["joined_artist"] == artists_to_look_for[i]], ignore_index=True)
  except KeyError: 
    pass

all_songs3 = all_songs3[(all_songs3['release_date'] >= "2016-01-01")]
all_songs3 = all_songs3.reset_index()
all_songs3.head()

,index,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year,main_artist,formatted_artist,joined_artist
0,7,0.00258,['Drake'],0.891,267067,0.628,0,0wwPcA6wtMf6HUMpIRdeP7,0.000190,2,0.0504,-7.863,1,Hotline Bling,76,2016-05-06,0.0551,134.966,0.552,2016,NaN,Drake,Drake
1,8,0.25600,['Drake'],0.809,298941,0.463,1,5mCPDVBb16L4XQwDdbRUpz,0.085000,11,0.1090,-11.377,1,Passionfruit,79,2017-03-18,0.0396,111.980,0.364,2017,NaN,Drake,Drake
2,9,0.11100,['Drake'],0.766,219692,0.283,1,6n3HGiq4v35D6eFOSwqYuo,0.017200,8,0.1150,-10.330,0,Teenage Fever,75,2017-03-18,0.0846,97.040,0.144,2017,NaN,Drake,Drake
3,10,0.03320,['Drake'],0.754,198973,0.449,1,6DCZcSspjsKoFjzjrWoCdn,0.000083,7,0.5520,-9.211,1,God's Plan,83,2018-06-29,0.1090,77.169,0.357,2018,NaN,Drake,Drake
4,11,0.01650,['Drake'],0.912,238614,0.412,1,0TlLq3lA83rQOYtrqBqSct,0.012600,7,0.1040,-8.074,1,Nonstop,78,2018-06-29,0.1230,154.983,0.423,2018,NaN,Drake,Drake


**Create a matrix of all the songs from all_songs3 df where each row contains the song features**

In [ ]:
s3_labels = []
for i in range(all_songs3.shape[0]):
  current = []
  for column in all_songs3:
    if (column == "acousticness" or column == "danceability" or column == "energy" 
        or column == "instrumentalness" or column == "liveness" or column == "loudness" 
        or column == "speechiness" or column == "tempo" or column == "valence" ):
      current.append(all_songs3[column][i])
  s3_labels.append(current)
print(len(s3_labels))

163


**Recommends 10 songs based on only the artists found in the spotify_user_playlist df**

In [ ]:
recommendations3 = []

temp = cosine_similarity(weighted_avg, s3_labels)
r3df = pd.DataFrame(temp).T # transpose row and comlums swap
top10 = r3df.nlargest(r3df.shape[0] ,0).index # index of top 10 closeest to songs in playlist
for index in top10:
  recommendations3.append("'" + all_songs3['name'][index] + "'" + " by " + all_songs3['artists'][index][2:-2])

chosenR3 = []
print("You should listen to: ")
i = 1
while len(chosenR3) < 10:
  ind = random.randint(0, r3df.shape[0])
  try:
    if recommendations3[ind] not in chosenR3:
      chosenR3.append(recommendations3[ind])
      print(i, ": " + recommendations3[ind])
      i += 1
  except IndexError:
    pass

You should listen to: 
1 : '20/20' by Lil Tjay
2 : 'WITHOUT YOU' by The Kid LAROI
3 : 'Dior' by Pop Smoke
4 : 'Attention' by The Weeknd
5 : 'Empty' by Juice WRLD
6 : 'In My Feelings' by Drake
7 : 'Secrets' by The Weeknd
8 : 'Used To' by Juice WRLD
9 : 'Maze' by Juice WRLD
10 : 'Man Of The Year' by Juice WRLD
